In [4]:
from scipy import signal as sg

print(sg.convolve([[255, 7, 3],
                  [212, 240, 4],
                  [218, 216, 230]], [[1, -1]], "valid")) # mode = full, same, valid

# same = pad zeros on borders so that the dimension does not change

[[-248   -4]
 [  28 -236]
 [  -2   14]]


In [8]:
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.convolve.html

# Only return the middle values of the convolution. 
print(sg.convolve([1, 2, 3], [0, 1, 0.5], 'same'))
# The two arrays are of the same length, so there is only one position where they completely overlap
print(sg.convolve([1, 2, 3], [0, 1, 0.5], 'valid'))

[ 1.   2.5  4. ]
[ 2.5]


In [9]:
print(sg.convolve([[1, 0, 0],
                  [0, 1, 0],
                  [0, 0, 1]], [[1, 1], [1, 1]], "same"))

[[1 1 0]
 [1 2 1]
 [0 1 2]]


In [15]:
print(sg.convolve([[1, 0],
                  [0, 1]], [[1, 1], [1, 1]], "same"))

import numpy as np

a = np.array([[1, 0],
           [0, 1]])
b = np.array([[1, 1], [1, 1]])
print(np.matmul(a, b.T))

[[1 1]
 [1 2]]
[[1 1]
 [1 1]]


In [33]:
from PIL import Image
import numpy as np

# read image
def np_from_img(fname):
    return np.asarray(Image.open(fname), dtype=np.float32)

# write image
def save_as_img(ar, fname):
    Image.fromarray(ar.round().astype(np.uint8)).save(fname)
    
# normalization
def norm(ar):
    return 255*np.absolute(ar)/np.max(ar)

# read image file
fname = './bridge.bmp'
img = np_from_img(fname)
print(img.shape)

# save blur images
save_as_img(norm(sg.convolve(img, [[1, 1, 1], [0, 0, 0], [-1, -1, -1]])), './portal-blur.png')

# sharpen
save_as_img(norm(sg.convolve(img, [[0, -1, 0], [-1, 6, -1], [0, -1, 0]])), './portal-sharpen.png')

# lighter
save_as_img(sg.convolve(img, [[0, 0, 0], [0, 2, 0], [0, 0, 0]]), './portal-lighter.png')

# save horizontal edges 
save_as_img(norm(sg.convolve(img, [[1.],[-1.]])), './portal-h.png')

# save vertical edges
save_as_img(norm(sg.convolve(img, [[1., -1.]])), './portal-v.png')


'''
img_name = 'bridge.bmp'
name, ext = img_name.split('.')
new_img_name = name + '_new.bmp'
print(new_img_name)
img_arr = np_from_img(img_name)
save_as_img(norm(img_arr), new_img_name)
'''

(512, 512)


"\nimg_name = 'bridge.bmp'\nname, ext = img_name.split('.')\nnew_img_name = name + '_new.bmp'\nprint(new_img_name)\nimg_arr = np_from_img(img_name)\nsave_as_img(norm(img_arr), new_img_name)\n"

In [134]:
# convolution 
# 2d convolution: 
#  - http://songho.ca/dsp/convolution/convolution2d_example.html
#  - http://www.songho.ca/dsp/convolution/convolution.html#convolution_2d
data = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
kernel = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

# kernel as filter
def convolve(data, kernel):
    h1, w1 = data.shape
    h2, w2 = kernel.shape
    
    out = np.zeros(data.shape)
    
    # indices for kernel centers
    cy, cx = np.divide(data.shape, 2).astype(int) # input.shape[0]/2, input.shape[1]/2
    
    for i in range(h1):
        for j in range(w1):
            for y in range(h2):
                for x in range(w2):
                    if i + y - cy >= 0 and i + y - cy < h1 and j + x - cx >= 0 and j + x - cx < w1:
                        out[i, j] += kernel[h2 -1 - y, w2 -1 - x] * data[i + y - cy, j + x - cx]
            
    return out
    
print(convolve(data, kernel))

[[-13. -20. -17.]
 [-18. -24. -18.]
 [ 13.  20.  17.]]


In [110]:
a = np.array([2, 2])
a = np.expand_dims(a, axis=1)
np.expand_dims(a, axis=0)

array([[[2],
        [2]]])

In [ ]:
'''

tf.nn.conv2d(input, filter, strides, padding, name)
 - input must be a 4d tensor [batch, in_height, in_width, in_channels]
 - filter/kernel shape: [filter_height, filter_width, in_channels, out_channels]
 - strides: 
 - padding:
 - name: 
 
Filter의 개수
 - 일반적으로 영상의 크기가 큰 입력단 근처에 있는 layer는 finite의 개수가 적고, 입력단에서 멀어질수록 filter의
 개수는 증가하는 경향이 있음
 - 각 레이어에서의 연사 시간/량을 비교적 일정하게 유지하여 시스템의 균형을 맞추게 필터의 개수를 조절
- 

Filter의 형태
 - 일반적으로 32*32 정도의 작은 크기의 입력 영상에 대해선 3x3, 5x5를 자주 사용
 - 7x7(1개) vs. 3*3 (3개)
   - 여러 개의 작은 크기의 필터를 중첩해서 사용하는 것이 좋음
   - 여러 개를 중첩하면 중간 단계에 있는 non-linearity를 활용하여 원하는 특징을 더 살릴 수 있음 (filter size smaller -> layer gets deeper)
   - 작은 필터를 여러 개 중첩해서 사용하는 것이 연산상 좋음
Stride 값
 - 크기를 조절한다는 면에서 생각할 때 pooling과 유사 (WxH smaller)
 - stride vs pooling? what's better? -> we don't know. It depends on the model you design.
Padding
 - 보통 convolution 연산을 하게 되면 경계 처리문제가 생김
   
   
'''

